In [1]:
import pandas as pd
import os
import librosa
import numpy as np
from IPython.display import Audio
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('metadata/UrbanSound8K.csv')

In [3]:
list1=[]
list2=[]
list3=[]
list4=[]
list5=[]
list6=[]
list7=[]
list8=[]
list9=[]
list10=[]

fold1 = 'audio/fold1'
list1 = os.listdir(fold1)

fold2 = 'audio/fold2'
list2 = os.listdir(fold2)

fold3 = 'audio/fold3'
list3 = os.listdir(fold3)

fold4 = 'audio/fold4'
list4 = os.listdir(fold4)

fold5 = 'audio/fold5'
list5 = os.listdir(fold5)

fold6 = 'audio/fold6'
list6 = os.listdir(fold6)

fold7 = 'audio/fold7'
list7 = os.listdir(fold7)

fold8 = 'audio/fold8'
list8 = os.listdir(fold8)

fold9 = 'audio/fold9'
list9 = os.listdir(fold9)

fold10 = 'audio/fold10'
list10 = os.listdir(fold10)

lista = list1 + list2 + list3 + list4 + list5 + list6 + list7 + list8 + list9 + list10

In [4]:
def path_class(filename):
    excerpt = df[df['slice_file_name'] == filename]
    
    if not excerpt.empty:
        path_name = os.path.join('audio', 'fold'+str(excerpt.fold.values[0]), filename)
        return path_name
    
    else:
        return None

In [5]:
path_list=[]
for i in lista:
    p = path_class(i)
    if (p!=None):
        path_list.append(p)

In [6]:
# Normalização min-max

def normalize(y):
    min_val = np.min(y)
    max_val = np.max(y)
    ynormalized = (y - min_val) / (max_val - min_val)
    return ynormalized

In [7]:
sr = 44100
hop_length=round(sr*0.0125)
window_length=round(sr*0.023)
n_fft=2**14
mfcc_time_size = 4*sr//hop_length+1

In [8]:
#Wave

def wave(y):
    ynormalized = normalize(y)
    return ynormalized

In [9]:
#MFCC 13

n_mfcc=13

def mfcc13(y,n_mfcc=13):
    mfcc = librosa.feature.mfcc(y=y,sr=sr,n_fft=n_fft,hop_length=hop_length,win_length=window_length,n_mfcc=n_mfcc)
    y_mfcc13 = normalize(mfcc)
    return y_mfcc13

In [10]:
#MFCC 40

n_mfcc=40

def mfcc40(y,n_mfcc=40):
    mfcc = librosa.feature.mfcc(y=y,sr=sr,n_fft=n_fft,hop_length=hop_length,win_length=window_length,n_mfcc=n_mfcc)
    y_mfcc40 = normalize(mfcc)
    return y_mfcc40

In [11]:
#MFCC delta

def mfcc_delta(y,n_mfcc=13):
    y_mfcc13 = librosa.feature.mfcc(y=y,sr=sr,n_fft=n_fft,hop_length=hop_length,win_length=window_length,n_mfcc=n_mfcc)
    mfcc_delta = librosa.feature.delta(y_mfcc13, width=3)
    y_mfcc_delta = normalize(mfcc_delta)
    
    return y_mfcc_delta

In [12]:
#MFCC delta 2

def mfcc_delta2(y, n_mfcc=13):
    y_mfcc13 = librosa.feature.mfcc(y=y,sr=sr,n_fft=n_fft,hop_length=hop_length,win_length=window_length,n_mfcc=n_mfcc)
    mfcc_delta2 = librosa.feature.delta(y_mfcc13, order=2, width=3)
    y_mfcc_delta2 = normalize(mfcc_delta2)
    
    return y_mfcc_delta2

In [13]:
#Spectrogram

def spectrogram(y):
    stft = librosa.core.stft(y, hop_length = hop_length, n_fft = n_fft)
    spectrogram = np.abs(stft)
    y_spectrogram = normalize(spectrogram)
    
    return y_spectrogram

In [14]:
#Logspectrogram

def logspectrogram(y):
    stft = librosa.core.stft(y, hop_length = hop_length, n_fft = n_fft)
    spectrogram = np.abs(stft)
    logspectrogram = librosa.amplitude_to_db(spectrogram)
    y_logspectrogram = normalize(logspectrogram)
    
    return y_logspectrogram

In [15]:
#Melspectrogram

def melspectrogram(y):
    stft = librosa.core.stft(y, hop_length = hop_length, n_fft = n_fft)
    spectrogram = np.abs(stft)
    melstft = librosa.feature.melspectrogram(S=spectrogram, sr=sr)
    y_melstft = normalize(melstft)
    
    return y_melstft

In [16]:
#Melspectrogram db

def melspectrogram_db(y):
    stft = librosa.core.stft(y, hop_length = hop_length, n_fft = n_fft)
    spectrogram = np.abs(stft)
    melstft = librosa.feature.melspectrogram(S=spectrogram, sr=sr)
    melstft_db = librosa.power_to_db(melstft, ref=np.max)
    y_melstft_db = normalize(melstft_db)
    
    return y_melstft_db

In [17]:
#Chromaspectrogram

def chromaspectrogram(y):
    stft = librosa.core.stft(y, hop_length = hop_length, n_fft = n_fft)
    spectrogram = np.abs(stft)
    chroma = librosa.feature.chroma_stft(y=y, S=spectrogram, sr=sr)
    y_chroma = normalize(chroma)
    
    return y_chroma

In [ ]:
#Data Frame

newdata = pd.DataFrame()
newdata['mfcc40'] = [np.zeros((40, mfcc_time_size)) for _ in range(len(path_list))]
newdata['mfcc13'] = [np.zeros((13, mfcc_time_size)) for _ in range(len(path_list))]
newdata['spectrogram'] = [np.empty((96001, 321), dtype=np.float32) for _ in range(len(path_list))]
newdata['logspectrogram'] = [np.empty((96001, 321), dtype=np.float32) for _ in range(len(path_list))]
newdata['melspectrogram'] = [np.empty((128, 321), dtype=np.float32) for _ in range(len(path_list))]
newdata['melspectrogram_db'] = [np.empty((128, 321), dtype=np.float32) for _ in range(len(path_list))]
newdata['melspectrogram_delta'] = [np.empty((13, 321), dtype=np.float32) for _ in range(len(path_list))]
newdata['melspectrogram_delta2'] = [np.empty((13, 321), dtype=np.float32) for _ in range(len(path_list))]
newdata['chromaspectrogram'] = [np.empty((12, 321), dtype=np.float32) for _ in range(len(path_list))]
i=0
for f in path_list:
    #print(f)
    f = f'{f}'
    y, previous_sr = librosa.load(f, sr=None)
    y_resample =librosa.resample(y, orig_sr=previous_sr, target_sr=sr)
    
    if len(y_resample)<4*sr:
        # Calculate the number of times to repeat the audio
        repeat = int(np.ceil(4 * sr / len(y_resample)))

        # Repeat the audio
        y_extended = np.tile(y_resample, repeat)

        # Trim the array to the desired duration
        y_extended = y_extended[:int(4 * sr)]

    else:
        y_extended = y_resample
    
    path_parts = f.split('/')
    arquivo = path_parts[-1]
    
    if arquivo in df['slice_file_name'].values:
        fold = df[df['slice_file_name'] == arquivo].iloc[0]['fold']
        salience = df[df['slice_file_name'] == arquivo].iloc[0]['salience']
        classID = df[df['slice_file_name'] == arquivo].iloc[0]['classID']
        

        newdata.at[i, 'file'] = arquivo
        newdata.at[i, 'salience'] = salience
        newdata.at[i, 'fold'] = fold
        newdata.at[i, 'classID'] = classID
                     
    
    # MFCCs 40
    #newdata.at[i, 'mfcc40'] = mfcc40(y_extended)

    # MFCCs 13
    #newdata.at[i, 'mfcc13'] = mfcc13(y_extended)
    
    #Spectrogram
    newdata.at[i, 'spectrogram'] = spectrogram(y_extended)
    
    #Log spectrogram
    #newdata.at[i, 'logspectrogram'] = logspectrogram(y_extended)

    #Chroma spectrogram
    #newdata.at[i, 'chromaspectrogram'] = chromaspectrogram(y_extended)

    #Mel spectrogram
    #newdata.at[i, 'melspectrogram'] = melspectrogram(y_extended)

    #Mel spectrogram DB
    #newdata.at[i, 'melspectrogram_db'] = melspectrogram_db(y_extended)

    #Mel spectrogram delta
    #newdata.at[i, 'melspectrogram_delta'] = mfcc_delta(y_extended)

    #Mel spectrogram delta ordem 2
    #newdata.at[i, 'melspectrogram_delta2'] = mfcc_delta2(y_extended)
        
    i+=1

columns = ['file', 'salience', 'fold', 'spectrogram', 'classID']

newdata = newdata[columns]

In [ ]:
#columns2 = ['logspectrogram', 'chromaspectrogram', 'melspectrogram', 'melspectrogram_db', 'melspectrogram_delta', 'melspectrogram_delta2']

#newdata = newdata[columns1 + columns2]

In [ ]:
newdata.to_json('features3.json')

In [ ]:
df1 = pd.read_json('features3.json')

In [ ]:
df1

In [ ]:
for i in range(0,8732):
    a = df1.loc[i]['mfcc40']
    b = np.array(a)
    if (b.shape!=(40,321)):
        print(b.shape)